<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/Hybrid%20RF-DNN%20for%204%20class%20classifications%20based%20on%20WSNBFSF%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid RF-DNN for 4 class classifications based on WSNBFSF dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# load dataset
dataset = pd.read_csv("dataset.csv")

### **Data preprocessing**

In [3]:
# Data preprocessing
y = dataset['Class'].values
X = dataset.drop(['Class'],axis=1)
X = X.values

In [5]:
# balance classes using SMOTE
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Hybrid combination of RF-DNN for 4 class classifications**

In [7]:
# train RF model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

RandomForestClassifier(random_state=42)

In [8]:
# train DNN model
input_dim = X.shape[1]
num_classes = len(np.unique(y))
dnn_model = Sequential()
dnn_model.add(Dense(128, input_dim=input_dim, activation='relu'))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dense(4, activation='softmax'))
dnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn_model.fit(X, pd.get_dummies(y), epochs=15, batch_size=32)

Epoch 1/15
32857/32857 [==============================] - 65s 2ms/step - loss: 0.3814 - accuracy: 0.8599
Epoch 2/15
32857/32857 [==============================] - 63s 2ms/step - loss: 0.1883 - accuracy: 0.9316
Epoch 3/15
32857/32857 [==============================] - 63s 2ms/step - loss: 0.1715 - accuracy: 0.9378
Epoch 4/15
32857/32857 [==============================] - 63s 2ms/step - loss: 0.1604 - accuracy: 0.9435
Epoch 5/15
32857/32857 [==============================] - 64s 2ms/step - loss: 0.1308 - accuracy: 0.9545
Epoch 6/15
32857/32857 [==============================] - 65s 2ms/step - loss: 0.1250 - accuracy: 0.9583
Epoch 7/15
32857/32857 [==============================] - 63s 2ms/step - loss: 0.1193 - accuracy: 0.9615
Epoch 8/15
32857/32857 [==============================] - 62s 2ms/step - loss: 0.1175 - accuracy: 0.9622
Epoch 9/15
32857/32857 [==============================] - 62s 2ms/step - loss: 0.1097 - accuracy: 0.9648
Epoch 10/15
32857/32857 [==============================

In [9]:
# combine RF and DNN models
def hybrid_model_predict(X):
    rf_pred = rf_model.predict(X)
    dnn_pred = np.argmax(dnn_model.predict(X), axis=-1)
    hybrid_pred = []
    for i in range(len(X)):
        if rf_pred[i] == dnn_pred[i]:
            hybrid_pred.append(rf_pred[i])
        else:
            hybrid_pred.append(np.random.choice([rf_pred[i], dnn_pred[i]]))
    return np.array(hybrid_pred)

# evaluate hybrid model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = hybrid_model_predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print('Hybrid model accuracy:', accuracy)

6572/6572 [==============================] - 9s 1ms/step
Hybrid model accuracy: 0.9790661067809264
